## First classification attempt using the around fornax sample as training set

I choose a region $2^o \times 2^o$, apply some cuts and find LSB / UDG candidates.

The region I select is:
\begin{equation}
53.61625 \leq \mbox{RA} \leq 55.61625\\
-36.44833 \leq \mbox{DEC} \leq -34.44833 
\end{equation}
 
I have found the UDGs etc

In [124]:
# Import stuff
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from astropy.io import fits
import skymapper as skm
from code.plot_utils import plot_pretty
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib import rcParams
rcParams['font.family'] = 'serif'

%matplotlib inline
%pylab inline 
plot_pretty() 

Populating the interactive namespace from numpy and matplotlib


In [125]:
REG_3 = fits.open('Region_3.fits')
print REG_3[1].header.keys

<bound method Header.keys of XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  104 / length of dimension 1                          
NAXIS2  =               178944 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   24 / number of table fields                         
TTYPE1  = 'ra      '                                                            
TFORM1  = 'D       '                                                            
TTYPE2  = 'dec     '                                                            
TFORM2  = 'D       '                                                            

In [126]:
# Import everything 
# ========================================================
# ========================================================
# spread_model_i and its error
spread_model_i = REG_3[1].data['spread_model_i']
speraderr_model_i = REG_3[1].data['spreaderr_model_i']

# RA and DEC 
RA = REG_3[1].data['ra']
DEC = REG_3[1].data['dec']

# Image A/ Image B
IMAGE_A = REG_3[1].data['a_image']
IMAGE_B = REG_3[1].data['b_image']

# Magnitudes 
MAG_AUTO_G = REG_3[1].data['mag_auto_g']
MAG_AUTO_R = REG_3[1].data['mag_auto_r']
MAG_AUTO_I = REG_3[1].data['mag_auto_i']

# Flux radii - convert from pixels to arcseconds
FLUX_RADIUS_G = 0.263*REG_3[1].data['flux_radius_g']
FLUX_RADIUS_R = 0.263*REG_3[1].data['flux_radius_r']
FLUX_RADIUS_I = 0.263*REG_3[1].data['flux_radius_i'] 

# ================================================================================
# ================================================================================
# SURFACE BRIGHTNESSES

##Effective model surface brightness (SB) above background [mag/sq. arcmin]. 
#SB at the isophote which includes half of the flux from the model, above background
MU_EFF_G = REG_3[1].data['mu_eff_model_g']
MU_EFF_R = REG_3[1].data['mu_eff_model_r']
MU_EFF_I = REG_3[1].data['mu_eff_model_i']

#Peak surface brightness above background [mag/asec^2]
MU_MAX_G = REG_3[1].data['mu_max_g']
MU_MAX_R = REG_3[1].data['mu_max_r']
MU_MAX_I = REG_3[1].data['mu_max_i']

#Peak surface brightness above background  - using Model[mag/asec^2]
MU_MAX_MODEL_G = REG_3[1].data['mu_max_model_g']
MU_MAX_MODEL_R = REG_3[1].data['mu_max_model_r']
MU_MAX_MODEL_I = REG_3[1].data['mu_max_model_i']

#Mean surface brightness using the whole area inside the isophote used for MU_EFF_MODEL
MU_MEAN_G = REG_3[1].data['mu_mean_model_g']
MU_MEAN_R = REG_3[1].data['mu_mean_model_r']
MU_MEAN_I = REG_3[1].data['mu_mean_model_i']


# Define ellipticity 
ellipticity = 1.0 - IMAGE_B/IMAGE_A

print(len(RA))

178944


### Perform cuts

I perform the following cuts in my data:

- Star-galaxy separation:
\begin{equation}
spread\_model\_i + (5/3)spreaderr\_model\_i > 0.007
\end{equation}


- Surface brightness cut:
\begin{equation} 
\bar{\mu}_{eff}(g) > 24.3 \,\, \mbox{mag}\,\mbox{arcsec}^{-2}
\end{equation}

- Color cuts:

\begin{equation}
- 0.1 < (g-i) < 1.4 \\
(g - r )>0.7\cdot(g-i) - 0.4 \\
(g - r) < 0.7 \cdot (g-i) + 0.4
\end{equation}

- Radius cut:

\begin{equation}
2''.5 < r_{1/2} < 20'',
\end{equation}

where: $\mbox{FLUX_RADIUS_I} \equiv r_{1/2}$.

- Ellipticity cut:

\begin{equation}
\epsilon = 1 - \frac{b}{a} < 0.6
\end{equation}

#### Define the cuts now

In [127]:
# Define the cuts here 
# ====================================================

# Star-galaxy separation 
star_gal_cut = ((spread_model_i+(5.0/3.0)*speraderr_model_i)>0.007)

# Surface brightness cut
mu_eff_cut = (MU_EFF_G > 24.3)

# Color cuts 
col_1 = ((MAG_AUTO_G - MAG_AUTO_I)>(-0.1))
col_2 = ((MAG_AUTO_G - MAG_AUTO_I)<(1.4))
col_3 = ((MAG_AUTO_G - MAG_AUTO_R) > 0.7*(MAG_AUTO_G - MAG_AUTO_I) - 0.4)
col_4 = ((MAG_AUTO_G - MAG_AUTO_R) < 0.7*(MAG_AUTO_G - MAG_AUTO_I) + 0.4)
color_cuts = col_1&col_2&col_3&col_4

# Radius cut
radius_cut = ((FLUX_RADIUS_I > 2.5)&(FLUX_RADIUS_I < 20.0))

# Ellipticity cut
ell_cut = (ellipticity < 0.6)

# Summarize the final cut

Total_cut = star_gal_cut&mu_eff_cut&color_cuts&radius_cut&ell_cut

#### Apply the cuts now

In [128]:


# RA and DEC
RA = RA[Total_cut]
DEC = DEC[Total_cut]

# Image A/ Image B
IMAGE_A = IMAGE_A[Total_cut]
IMAGE_B = IMAGE_B[Total_cut]

#Magnitudes 
MAG_AUTO_G = MAG_AUTO_G[Total_cut]
MAG_AUTO_R = MAG_AUTO_R[Total_cut]
MAG_AUTO_I = MAG_AUTO_I[Total_cut]

#Flux radii 
FLUX_RADIUS_G = FLUX_RADIUS_G[Total_cut]
FLUX_RADIUS_R = FLUX_RADIUS_R[Total_cut]
FLUX_RADIUS_I = FLUX_RADIUS_I[Total_cut]

# ================================================================================
# ================================================================================
# SURFACE BRIGHTNESSES

##Effective model surface brightness (SB) above background [mag/sq. arcmin]. 
#SB at the isophote which includes half of the flux from the model, above background
MU_EFF_G = MU_EFF_G[Total_cut]
MU_EFF_R = MU_EFF_R[Total_cut]
MU_EFF_I = MU_EFF_I[Total_cut]

#Peak surface brightness above background [mag/asec^2]
MU_MAX_G = MU_MAX_G[Total_cut]
MU_MAX_R = MU_MAX_R[Total_cut]
MU_MAX_I = MU_MAX_I[Total_cut]

#Peak surface brightness above background  - using Model[mag/asec^2]
MU_MAX_MODEL_G = MU_MAX_MODEL_G[Total_cut]
MU_MAX_MODEL_R = MU_MAX_MODEL_R[Total_cut]
MU_MAX_MODEL_I = MU_MAX_MODEL_I[Total_cut]

#Mean surface brightness using the whole area inside the isophote used for MU_EFF_MODEL
MU_MEAN_G = MU_MEAN_G[Total_cut]
MU_MEAN_R = MU_MEAN_R[Total_cut]
MU_MEAN_I = MU_MEAN_I[Total_cut]

# Ellipticity
ellipticity = ellipticity[Total_cut]

print(len(RA))

321


Now let's choose the features we are going to use in the classification.

I will use:

- The magnitudes G, R, I
- Colors g-r, g-i, i - r
- Flux radii G,R,I
- Effective surface brightness G,R,I
- Maximum surface brightness G,R,I
- Ellipticity 


In [129]:
# Define the colors
col_g_r = MAG_AUTO_G - MAG_AUTO_R
col_g_i = MAG_AUTO_G - MAG_AUTO_I
col_i_r = MAG_AUTO_G - MAG_AUTO_R



# Define the train data matrix of features
X_feat_train = np.zeros([len(RA),16])
X_feat_train[:,0] = MAG_AUTO_G;X_feat_train[:,1] = MAG_AUTO_R;X_feat_train[:,2] = MAG_AUTO_I
X_feat_train[:,3] = col_g_r;X_feat_train[:,4] = col_g_i;X_feat_train[:,5] = col_i_r
X_feat_train[:,6] = FLUX_RADIUS_G;X_feat_train[:,7] = FLUX_RADIUS_R;X_feat_train[:,8] = FLUX_RADIUS_I
X_feat_train[:,9] = MU_EFF_G;X_feat_train[:,10] = MU_EFF_R;X_feat_train[:,11] = MU_EFF_I
X_feat_train[:,12] = MU_MAX_G; X_feat_train[:,13] =  MU_MAX_R;X_feat_train[:,14] =  MU_MAX_I
X_feat_train[:,15] = ellipticity 

#### Define the target classes array now

In [130]:
y = np.zeros(len(RA))
y[0]=1;y[2]=1;y[3]=1;y[9]=1;y[10]=1;y[12]=1
y[27]=1;y[34]=1;y[35]=1;y[36]=1;y[37]=1;y[40]=1
y[45]=1;y[46]=1;y[50]=1;y[55]=1;y[59]=1;y[67]=1
y[73]=1;y[74]=1;y[77]=1;y[81]=1;y[83]=1;y[85]=1
y[86]=1;y[87]=1;y[89]=1;y[90]=1;y[95]=1;y[96]=1
y[97]=1;y[100]=1;y[102]=1;y[104]=1;y[105]=1;y[107]=1
y[113]=1;y[116]=1;y[117]=1;y[121]=1;y[129]=1;y[130]=1
y[137]=1;y[139]=1;y[142]=1;y[147]=1;y[148]=1;y[151]=1
y[153]=1;y[154]=1;y[156]=1;y[157]=1;y[162]=1;y[167]=1
y[170]=1;y[178]=1;y[180]=1;y[182]=1;y[184]=1;y[186]=1
y[188]=1;y[190]=1;y[197]=1;y[199]=1;y[205]=1;y[206]=1
y[207]=1;y[208]=1;y[214]=1;y[215]=1;y[217]=1;y[218]=1
y[223]=1;y[228]=1;y[233]=1;y[243]=1;y[245]=1;y[256]=1
y[271]=1;y[277]=1;y[279]=1;y[283]=1;y[285]=1;y[287]=1
y[289]=1;y[290]=1;y[291]=1;y[292]=1;y[297]=1;y[299]=1
y[303]=1;y[307]=1;y[309]=1;y[312]=1;y[314]=1;y[315]=1
y[320]=1

In [131]:
# Import support vector machines
import sklearn
from sklearn.svm import SVC
svc_first = SVC(C=0.5,kernel='rbf')

In [132]:
# Fit the data now
svc_first.fit(X_feat_train, y)

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [133]:
svc_first.score(X_feat_train,y)

0.9190031152647975

Let's apply that to new cases

In [134]:
REG_4 = fits.open('Region_1.fits')

In [135]:
# Import everything 
# ========================================================
# ========================================================
# spread_model_i and its error
spread_model_i = REG_4[1].data['spread_model_i']
speraderr_model_i = REG_4[1].data['spreaderr_model_i']

# RA and DEC 
RA = REG_4[1].data['ra']
DEC = REG_4[1].data['dec']

# Image A/ Image B
IMAGE_A = REG_4[1].data['a_image']
IMAGE_B = REG_4[1].data['b_image']

# Magnitudes 
MAG_AUTO_G = REG_4[1].data['mag_auto_g']
MAG_AUTO_R = REG_4[1].data['mag_auto_r']
MAG_AUTO_I = REG_4[1].data['mag_auto_i']

# Flux radii - convert from pixels to arcseconds
FLUX_RADIUS_G = 0.263*REG_4[1].data['flux_radius_g']
FLUX_RADIUS_R = 0.263*REG_4[1].data['flux_radius_r']
FLUX_RADIUS_I = 0.263*REG_4[1].data['flux_radius_i'] 

# ================================================================================
# ================================================================================
# SURFACE BRIGHTNESSES

##Effective model surface brightness (SB) above background [mag/sq. arcmin]. 
#SB at the isophote which includes half of the flux from the model, above background
MU_EFF_G = REG_4[1].data['mu_eff_model_g']
MU_EFF_R = REG_4[1].data['mu_eff_model_r']
MU_EFF_I = REG_4[1].data['mu_eff_model_i']

#Peak surface brightness above background [mag/asec^2]
MU_MAX_G = REG_4[1].data['mu_max_g']
MU_MAX_R = REG_4[1].data['mu_max_r']
MU_MAX_I = REG_4[1].data['mu_max_i']

#Peak surface brightness above background  - using Model[mag/asec^2]
MU_MAX_MODEL_G = REG_4[1].data['mu_max_model_g']
MU_MAX_MODEL_R = REG_4[1].data['mu_max_model_r']
MU_MAX_MODEL_I = REG_4[1].data['mu_max_model_i']

#Mean surface brightness using the whole area inside the isophote used for MU_EFF_MODEL
MU_MEAN_G = REG_4[1].data['mu_mean_model_g']
MU_MEAN_R = REG_4[1].data['mu_mean_model_r']
MU_MEAN_I = REG_4[1].data['mu_mean_model_i']


# Define ellipticity 
ellipticity = 1.0 - IMAGE_B/IMAGE_A

print(len(RA))

53513


In [136]:
# Define the cuts here 
# ====================================================

# Star-galaxy separation 
star_gal_cut = ((spread_model_i+(5.0/3.0)*speraderr_model_i)>0.007)

# Surface brightness cut
mu_eff_cut = (MU_EFF_G > 24.3)

# Color cuts 
col_1 = ((MAG_AUTO_G - MAG_AUTO_I)>(-0.1))
col_2 = ((MAG_AUTO_G - MAG_AUTO_I)<(1.4))
col_3 = ((MAG_AUTO_G - MAG_AUTO_R) > 0.7*(MAG_AUTO_G - MAG_AUTO_I) - 0.4)
col_4 = ((MAG_AUTO_G - MAG_AUTO_R) < 0.7*(MAG_AUTO_G - MAG_AUTO_I) + 0.4)
color_cuts = col_1&col_2&col_3&col_4

# Radius cut
radius_cut = ((FLUX_RADIUS_I > 2.5)&(FLUX_RADIUS_I < 20.0))

# Ellipticity cut
ell_cut = (ellipticity < 0.6)

# Summarize the final cut

Total_cut = star_gal_cut&mu_eff_cut&color_cuts&radius_cut&ell_cut

# RA and DEC
RA = RA[Total_cut]
DEC = DEC[Total_cut]

# Image A/ Image B
IMAGE_A = IMAGE_A[Total_cut]
IMAGE_B = IMAGE_B[Total_cut]

#Magnitudes 
MAG_AUTO_G = MAG_AUTO_G[Total_cut]
MAG_AUTO_R = MAG_AUTO_R[Total_cut]
MAG_AUTO_I = MAG_AUTO_I[Total_cut]

#Flux radii 
FLUX_RADIUS_G = FLUX_RADIUS_G[Total_cut]
FLUX_RADIUS_R = FLUX_RADIUS_R[Total_cut]
FLUX_RADIUS_I = FLUX_RADIUS_I[Total_cut]

# ================================================================================
# ================================================================================
# SURFACE BRIGHTNESSES

##Effective model surface brightness (SB) above background [mag/sq. arcmin]. 
#SB at the isophote which includes half of the flux from the model, above background
MU_EFF_G = MU_EFF_G[Total_cut]
MU_EFF_R = MU_EFF_R[Total_cut]
MU_EFF_I = MU_EFF_I[Total_cut]

#Peak surface brightness above background [mag/asec^2]
MU_MAX_G = MU_MAX_G[Total_cut]
MU_MAX_R = MU_MAX_R[Total_cut]
MU_MAX_I = MU_MAX_I[Total_cut]

#Peak surface brightness above background  - using Model[mag/asec^2]
MU_MAX_MODEL_G = MU_MAX_MODEL_G[Total_cut]
MU_MAX_MODEL_R = MU_MAX_MODEL_R[Total_cut]
MU_MAX_MODEL_I = MU_MAX_MODEL_I[Total_cut]

#Mean surface brightness using the whole area inside the isophote used for MU_EFF_MODEL
MU_MEAN_G = MU_MEAN_G[Total_cut]
MU_MEAN_R = MU_MEAN_R[Total_cut]
MU_MEAN_I = MU_MEAN_I[Total_cut]

# Ellipticity
ellipticity = ellipticity[Total_cut]

print(len(RA))

24


In [137]:
# Define the colors
col_g_r = MAG_AUTO_G - MAG_AUTO_R
col_g_i = MAG_AUTO_G - MAG_AUTO_I
col_i_r = MAG_AUTO_G - MAG_AUTO_R



# Define the train data matrix of features
X_feat_test = np.zeros([len(RA),16])
X_feat_test[:,0] = MAG_AUTO_G;X_feat_test[:,1] = MAG_AUTO_R;X_feat_test[:,2] = MAG_AUTO_I
X_feat_test[:,3] = col_g_r;X_feat_test[:,4] = col_g_i;X_feat_test[:,5] = col_i_r
X_feat_test[:,6] = FLUX_RADIUS_G;X_feat_test[:,7] = FLUX_RADIUS_R;X_feat_test[:,8] = FLUX_RADIUS_I
X_feat_test[:,9] = MU_EFF_G;X_feat_test[:,10] = MU_EFF_R;X_feat_test[:,11] = MU_EFF_I
X_feat_test[:,12] = MU_MAX_G; X_feat_test[:,13] =  MU_MAX_R;X_feat_test[:,14] =  MU_MAX_I
X_feat_test[:,15] = ellipticity 

In [138]:
# Let's predict now

y_pred = svc_first.predict(X_feat_test)

In [139]:
for i in range(len(RA)):
    print y_pred[i], ',', RA[i],',',DEC[i]

1.0 , 43.277539 , -42.638234
0.0 , 44.043769 , -42.699666
0.0 , 44.170142 , -42.774577
0.0 , 43.962621 , -43.264281
0.0 , 44.165595 , -42.789356
0.0 , 43.53207 , -42.938268
0.0 , 43.532987 , -42.939046
0.0 , 43.30042 , -42.324723
0.0 , 43.991357 , -42.782793
0.0 , 43.922765 , -42.811271
0.0 , 43.990666 , -42.783398
0.0 , 44.103335 , -43.290718
0.0 , 44.062772 , -42.729035
0.0 , 43.994055 , -42.790393
1.0 , 43.277222 , -42.451647
1.0 , 43.40656 , -43.146752
1.0 , 43.458562 , -42.399784
0.0 , 43.555758 , -42.679232
1.0 , 43.213421 , -42.376037
1.0 , 43.976675 , -42.331759
0.0 , 43.805092 , -42.688145
1.0 , 43.682633 , -42.998054
0.0 , 43.747915 , -42.621132
0.0 , 43.828257 , -42.622704
